# 🔍 GHRepoLens: GitHub Repository Analysis Tool

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alaamer12/GHRepoLens/blob/main/examples/colab_example.ipynb)

## 📝 Overview
GHRepoLens is a powerful tool for analyzing GitHub repositories. It provides detailed insights about:
- Code quality metrics
- Activity and engagement
- Repository statistics
- Visual analytics

This notebook provides an interactive interface to use GHRepoLens with different configurations and analysis modes.

## 🚀 Setup

First, we'll clone the repository and install dependencies:

In [ ]:
#@title Clone Repository and Install Dependencies
!git clone https://github.com/alaamer12/GHRepoLens.git
%cd GHRepoLens
!pip install -r requirements.txt

## 🔑 Authentication Setup

GHRepoLens supports multiple authentication methods. Choose your preferred method below:

In [ ]:
#@title Authentication Method
AUTH_METHOD = "Direct Input" #@param ["Direct Input", "Environment Variables", ".env File", "Config File"]

if AUTH_METHOD == "Direct Input":
    GITHUB_TOKEN = "" #@param {type:"string"}
    GITHUB_USERNAME = "" #@param {type:"string"}
    
    import os
    os.environ["GITHUB_TOKEN"] = GITHUB_TOKEN
    os.environ["GITHUB_USERNAME"] = GITHUB_USERNAME
    
elif AUTH_METHOD == ".env File":
    %%writefile .env
    GITHUB_TOKEN=your_token_here
    GITHUB_USERNAME=your_username_here
    
    import dotenv
    dotenv.load_dotenv()
    
elif AUTH_METHOD == "Config File":
    %%writefile config.ini
    [github]
    token = your_token_here
    username = your_username_here
    
# Validate credentials
from github import Github
try:
    g = Github(os.environ.get("GITHUB_TOKEN"))
    user = g.get_user()
    print(f"✅ Successfully authenticated as {user.login}")
except Exception as e:
    print(f"❌ Authentication failed: {str(e)}")

## 📊 Analysis Configuration

Configure your analysis parameters below:

In [ ]:
#@title Basic Configuration
ANALYSIS_MODE = "full" #@param ["full", "demo", "test", "quicktest"] {type:"string"}
REPOSITORY_VISIBILITY = "all" #@param ["all", "public", "private"] {type:"string"}
INCLUDE_ORGANIZATIONS = False #@param {type:"boolean"}
ORGANIZATION_NAMES = "" #@param {type:"string"}
ZIP_FILENAME = "ghrepolens_reports.zip" #@param {type:"string"}
REPORTS_DIR = "reports" #@param {type:"string"}

print(f"📝 Selected Mode: {ANALYSIS_MODE}")
print(f"🔒 Repository Visibility: {REPOSITORY_VISIBILITY}")

if INCLUDE_ORGANIZATIONS:
    org_list = [org.strip() for org in ORGANIZATION_NAMES.split(",") if org.strip()]
    if org_list:
        print(f"🏢 Including organizations: {', '.join(org_list)}")
    else:
        print("⚠️ No valid organization names provided")

In [ ]:
#@title Advanced Configuration
USE_CUSTOM_CONFIG = False #@param {type:"boolean"}
MAX_WORKERS = 4 #@param {type:"slider", min:1, max:8, step:1}
INACTIVE_THRESHOLD_DAYS = 180 #@param {type:"integer"}
ENABLE_CHECKPOINTING = True #@param {type:"boolean"}

if USE_CUSTOM_CONFIG:
    config = f"""
    [analysis]
    reports_dir = reports
    clone_dir = temp_repos
    max_workers = {MAX_WORKERS}
    inactive_threshold_days = {INACTIVE_THRESHOLD_DAYS}
    large_repo_loc_threshold = 1000

    [filters]
    skip_forks = false
    skip_archived = false
    include_private = true
    analyze_clones = false

    [checkpointing]
    enable_checkpointing = {str(ENABLE_CHECKPOINTING).lower()}
    checkpoint_file = github_analyzer_checkpoint.pkl
    checkpoint_threshold = 100
    """
    
    with open("config.ini", "w") as f:
        f.write(config)
    print("✅ Custom configuration saved to config.ini")

## 🚀 Run Analysis

Now let's run the repository analysis with your configuration:

In [ ]:
import asyncio
from main import run_analysis

async def main():
    try:
        await run_analysis(
            token=os.environ["GITHUB_TOKEN"],
            username=os.environ["GITHUB_USERNAME"],
            mode=ANALYSIS_MODE,
            config_file="config.ini" if USE_CUSTOM_CONFIG else None,
            include_orgs=org_list if INCLUDE_ORGANIZATIONS else None,
            visibility=REPOSITORY_VISIBILITY
        )
    except Exception as e:
        print(f"❌ Analysis failed: {str(e)}")
        raise

await main()

## 📈 Results & Visualization

Display the analysis results and generated reports:

In [ ]:
import glob
import os

# Display generated reports
print("Generated Reports:")
for file in glob.glob("reports/*"):
    print(f"📄 {os.path.basename(file)}")

## 📥 Download Reports

Download all generated reports in a zip file:

In [ ]:
import zipfile
import os
from google.colab import files



if os.path.exists(REPORTS_DIR) and os.listdir(REPORTS_DIR):
    with zipfile.ZipFile(ZIP_FILENAME, 'w') as zip_file:
        for root, dirs, filenames in os.walk(REPORTS_DIR):
            for filename in filenames:
                file_path = os.path.join(root, filename)
                arcname = os.path.relpath(file_path, REPORTS_DIR)
                zip_file.write(file_path, arcname=arcname)
                print(f"📎 Added {filename} to {ZIP_FILENAME}")
    
    # Download the zip file
    print(f"\n⬇️ Downloading {ZIP_FILENAME}...")
    files.download(ZIP_FILENAME)
else:
    print("❌ No reports found in reports directory")

---
## 📝 Credits

**GHRepoLens** is developed by [Alaamer](https://github.com/alaamer12).

- Version: 0.5.0
- License: MIT
- Homepage: [github.com/alaamer12/GHRepoLens](https://github.com/alaamer12/GHRepoLens)
- Bug Reports: [github.com/alaamer12/GHRepoLens/issues](https://github.com/alaamer12/GHRepoLens/issues)

For more information about using GHRepoLens, check out our [documentation](https://github.com/alaamer12/GHRepoLens#readme).

*Built with ❤️ for the GitHub community*